In [24]:
import os
import sys
import urllib.request
import json
import pandas as pd
from bs4 import BeautifulSoup

# 계획

- 크롤링 준비
    - 국회의원이름리스트MPnames 만들기
        - 중복이 없는지 확인, 있다면 그 국회의원에 한해 신뢰도가 떨어지겠거니 일단 생각만 해둔다.
    - getNewsAbout(MPname)이름 넣으면 크롤링 데이터 반환하는 함수 만들기
        - return df_org (컬럼 구성 : ['title', 'originallink', 'link', 'description', 'pubDate'])
    - reduction1(df) 데이터 프레임을 넣으면 1차 전처리를 해서 데이터 프레임 반환하는 함수 만들기
        - return df_r1 (위와 같음)
    - reduction2(df) 데이터 프레임을 넣으면 2차 전처리를 해서 문자열 반환하는 함수 만들기
        - return text

- 크롤링
    - 만든거 for 문으로 휘리릭
    - 과정과 결과를 딕셔너리  에 저장 {MPname: [df_org, df_r1, text]}

- 1차 전처리
    - 한글이외의 것 삭제
    - 검색어 삭제 (여기선 윤석열)
    - (title, description)으로 중복 거른다.

- 2차 전처리
    - 모든 title description을 하나의 문자열 text로 합친다.

# 구현

## 함수,, 만들기
### MPnames 
- 국회의원이름리스트 만들기
- 중복이 없는지 확인, 있다면 그 국회의원에 한해 신뢰도가 떨어지겠거니 일단 생각만 해둔다.

In [64]:
MP = pd.read_pickle('MP.pkl')

In [65]:
MP.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   의원명     299 non-null    object
 1   정당      299 non-null    object
 2   소속위원회   298 non-null    object
 3   지역      299 non-null    object
 4   성별      299 non-null    object
 5   당선횟수    299 non-null    object
 6   당선방법    299 non-null    object
dtypes: object(7)
memory usage: 16.5+ KB


In [76]:
MPnames = MP['의원명']

In [94]:
MPnames.count(), len(MPnames.unique())

(299, 297)

- 동명이인이 있다.

- 김병욱(33, 34), 이수진(191, 192)

In [86]:
MP[MP.duplicated(subset=['의원명'], keep='last')]

,의원명,정당,소속위원회,지역,성별,당선횟수,당선방법
33,김병욱,더불어민주당,[국토교통위원회],경기 성남시분당구을,남,재선,지역구
191,이수진,더불어민주당,"[기획재정위원회, 예산결산특...",서울 동작구을,여,초선,지역구


In [87]:
MP[MP.duplicated(subset=['의원명'], keep='first')]

,의원명,정당,소속위원회,지역,성별,당선횟수,당선방법
34,김병욱,국민의힘,"[교육위원회, 예산결산특별위원회]",경북 포항시남구울릉군,남,초선,지역구
192,이수진,더불어민주당,"[민생경제안정특별위원회, 국...",비례대표,여,초선,비례대표


In [99]:
for name in MPnames:
    print(name, end=" ")

강기윤 강대식 강득구 강민국 강민정 강병원 강선우 강은미 강준현 강훈식 고민정 고영인 고용진 구자근 권명호 권성동 권영세 권은희 권인숙 권칠승 기동민 김경만 김경협 김교흥 김기현 김남국 김도읍 김두관 김미애 김민기 김민석 김민철 김병기 김병욱 김병욱 김병주 김상훈 김상희 김석기 김선교 김성원 김성주 김성환 김수흥 김승남 김승수 김승원 김영배 김영선 김영식 김영주 김영진 김영호 김예지 김용민 김용판 김웅 김원이 김윤덕 김의겸 김정재 김정호 김종민 김주영 김진표 김철민 김태년 김태호 김학용 김한규 김한정 김형동 김홍걸 김회재 김희곤 김희국 남인순 노용호 노웅래 도종환 류성걸 류호정 맹성규 문정복 문진석 민병덕 민형배 민홍철 박광온 박대수 박대출 박덕흠 박범계 박병석 박상혁 박성민 박성준 박성중 박수영 박영순 박완주 박용진 박재호 박정 박정하 박주민 박진 박찬대 박형수 박홍근 배준영 배진교 배현진 백종헌 백혜련 변재일 서동용 서범수 서병수 서삼석 서영교 서영석 서일준 서정숙 설훈 성일종 소병철 소병훈 송갑석 송기헌 송석준 송언석 송옥주 송재호 신동근 신영대 신원식 신정훈 신현영 심상정 안규백 안민석 안병길 안철수 안호영 양경숙 양금희 양기대 양이원영 양정숙 양향자 어기구 엄태영 오기형 오영환 용혜인 우상호 우원식 위성곤 유경준 유기홍 유동수 유상범 유의동 유정주 윤건영 윤관석 윤두현 윤미향 윤상현 윤영덕 윤영석 윤영찬 윤재갑 윤재옥 윤주경 윤준병 윤창현 윤한홍 윤호중 윤후덕 이개호 이달곤 이동주 이만희 이명수 이병훈 이상민 이상헌 이성만 이소영 이수진 이수진 이양수 이용 이용빈 이용선 이용우 이용호 이원욱 이원택 이은주 이인선 이인영 이장섭 이재명 이재정 이정문 이종배 이종성 이주환 이채익 이철규 이탄희 이태규 이학영 이해식 이헌승 이형석 인재근 임병헌 임오경 임이자 임종성 임호선 장경태 장동혁 장제원 장철민 장혜영 전봉민 전용기 전재수 전주혜 전해철 전혜숙 정경희 정동만 정성호 정우택 정운천 정일영 정점식 정진석 정찬민 정청래 정춘숙 정태호 정필모 정희용 조경태 

### getNewsAbout(name)
- 이름 넣으면 크롤링 데이터 반환하는 함수 만들기
- return df_org (컬럼 구성 : ['title', 'originallink', 'link', 'description', 'pubDate']) 

- https://developers.naver.com/docs/serviceapi/search/news/news.md#%EB%89%B4%EC%8A%A4

In [258]:
df = pd.DataFrame()

In [260]:
df.empty

True

In [277]:
client_id = "ISdXTGuV8Tdqx9fNxPgz"
client_secret = "e2kcexi37t"
display = 100 # 출력건수 (최대)
sort = 'sim' # 유사도

def getNewsAbout(name):    
    encText = urllib.parse.quote(name)
    url = f'https://openapi.naver.com/v1/search/news?query={encText}&display={display}&sort={sort}'

    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)

    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    
    df = pd.DataFrame()
    if(rescode==200):
        response_body = response.read()
#         print('getNewsAbout : crawling about ', name, 'done.')
    else:
        print('getNewsAbout : crawling about ', name, 'failed.')
        print("Error Code:" + rescode)
        return df
        
    df = response_body.decode('utf-8')
    df = json.loads(df)
    df = pd.DataFrame(df['items'])
    
    return df

In [231]:
name = MPnames[0]
df = getNewsAbout(name)

Success


In [232]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         100 non-null    object
 1   originallink  100 non-null    object
 2   link          100 non-null    object
 3   description   100 non-null    object
 4   pubDate       100 non-null    object
dtypes: object(5)
memory usage: 4.0+ KB


In [233]:
df.head()

,title,originallink,link,description,pubDate
0,<b>강기윤</b> &quot...,http://www.edail...,https://n.news.n...,<b>강기윤</b> 국민의힘 ...,"Fri, 29 Jul 2022..."
1,<b>강기윤</b> 의원 “코...,https://mdtoday....,https://mdtoday....,▲ 국민의힘 <b>강기윤</b...,"Fri, 29 Jul 2022..."
2,보건복지위원장 정춘숙…간사 <...,https://www.dail...,https://www.dail...,﻿﻿﻿사진설명 = 좌측부터 보...,"Fri, 22 Jul 2022..."
3,후반기 국회 복지위원장에 정춘...,http://www.medig...,http://www.medig...,여당 간사는 국민의힘 <b>강...,"Fri, 22 Jul 2022..."
4,&quot;팬데믹 시대 과학 ...,http://www.sisao...,http://www.sisao...,토론자: <b>강기윤</b>-...,"Tue, 26 Jul 2022..."


### reduction1(df, name) 
- 데이터 프레임을 넣으면 1차 전처리를 해서 데이터 프레임 반환하는 함수 만들기
- return df_r1 (위와 같음)

- 1차 전처리
    - (title, description)에 대하여
        - 한글이외의 것 삭제
        - 검색어 삭제 (여기선 윤석열)
    - (title, description)으로 중복 거른다.

In [162]:
import re

In [163]:
df.info(), type(df.title[0]), type(df.description[0])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         100 non-null    object
 1   originallink  100 non-null    object
 2   link          100 non-null    object
 3   description   100 non-null    object
 4   pubDate       100 non-null    object
dtypes: object(5)
memory usage: 4.0+ KB


(None, str, str)

In [308]:
def reduction1(df, name):
    
    if df.empty:
        print("reduction1 : empty data.")
        return df
    
    df.title = df.title.apply(lambda x : re.sub('[^가-힣]+', ' ', x))
    df.description = df.description.apply(lambda x : re.sub('[^가-힣]+', ' ', x))
    
    df.title = df.title.apply(lambda x : re.sub('\s+', ' ', x))
    df.description = df.description.apply(lambda x : re.sub('\s', ' ', x))
    
    df.title = df.title.apply(lambda x : x.replace(name, ''))
    df.description = df.description.apply(lambda x : x.replace(name, ''))
    
    df.drop_duplicates(subset=['title'], inplace=True)
    df.drop_duplicates(subset=['description'], inplace=True)
    
#     print("reduction1 done.")
     
    return df
df1 = df.copy()
df1.info()
df_r1 = reduction1(df1, name)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88 entries, 0 to 99
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         88 non-null     object
 1   originallink  88 non-null     object
 2   link          88 non-null     object
 3   description   88 non-null     object
 4   pubDate       88 non-null     object
dtypes: object(5)
memory usage: 4.1+ KB


In [309]:
df_r1.head()

,title,originallink,link,description,pubDate
0,박찬호 양천소방서장 국회의원과...,http://fpn119.co...,http://fpn119.co...,양천소방서 서장 박찬호 는 ...,"Fri, 29 Jul 2022..."
1,발언하는 의원,http://www.newsi...,https://n.news.n...,더불어민주당 서해 공무원 사망...,"Wed, 20 Jul 2022..."
2,뱀사골 가는 길 우뚝 선 천년...,http://www.munhw...,https://n.news.n...,광한루의 시작은 정승이다 그가...,"Thu, 28 Jul 2022..."
3,속보 경찰 의원 대가성 후원...,http://www.kado....,https://n.news.n...,문재인 정부 시절 문화체육관광...,"Wed, 13 Jul 2022..."
4,항의하는 김병주 위원장 위원,http://www.newsp...,http://www.newsp...,일 서울 종로구 정부서울청사...,"Wed, 20 Jul 2022..."


### reduction2(df) 
- 데이터 프레임을 넣으면 2차 전처리를 해서 문자열 반환하는 함수 만들기
- return text

- 모든 title description을 하나의 문자열 text로 합친다.

In [225]:
df_r1_ = df_r1.copy()
df_r1_.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89 entries, 0 to 99
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         89 non-null     object
 1   originallink  89 non-null     object
 2   link          89 non-null     object
 3   description   89 non-null     object
 4   pubDate       89 non-null     object
dtypes: object(5)
memory usage: 6.2+ KB


In [322]:
def reduction2(df):
    if df.empty:
        print("reduction2 : empty data.")
        return ""
    text = ""
    sr = df.title + df.description
    text = " ".join(sr)
    text = re.sub('\s+', ' ', text)
    text = text.strip()
    
#     print("reduction2 done.")
    return text
text = reduction2(df_r1_)
print(len(text), type(text))
print(text[:1000])

10636 <class 'str'>
확진자 접촉 무증상자도 신속항원 검사 무료화 국민의힘 의원 사진 은 이르면 다음주부터 코로나 확진자와 접촉한 무증상자 또한 의료기관 의원은 코로나 확진자와 접촉한 무증상자에 대해서도 의사의 판단 하에 신속항원검사비를 무료로 의원 코로나 무증상자도 신속항원검사 무료로 받을 수 있어 국민의힘 의원 사진 의원실 제공 이르면 다음주부터 코로나 확진자와 접촉한 무증상자 또한 의료기관 등에서 의사의 판단 하에 신속항원검사 를 무료로 받게 된다 국회 보건복지위원회 소속 보건복지위원장 정춘숙 간사 강훈식 의원 사진설명 좌측부터 보건복지위원장 정춘숙 의원 국민의힘 의원 더민주당 강훈식 의원 보건복지위 위원장은 보건복지위원장 정춘숙 의원이 맡았으며 간사는 여당에서 의원 국민의힘 후반기 국회 복지위원장에 정춘숙 의원 양당 간사는 강훈식여당 간사는 국민의힘 의원이 야당 간사는 더불어민주당 강훈식 의원이 맡게됐다 의원은 전반기에 이어 후반기에도 간사 직책을 이어가게 됐으며 강훈식 의원은 전반기 팬데믹 시대 과학 방역의 핵심 관료 아닌 전문가에 맡길 것 안철수 민 당 토론자 박향 정재훈 조동찬 국민의힘 안철수 의원이 일 오전 국회 의원회관에서 열린 위기를 연합뉴스 제언 전문가 의사 결정 따른 과학방역 이뤄야 국가 책임 중요 국민의힘 의원은 코로나 재유행대비 의료체계 점검 긴급 간담회 연다 의원 국회 국민의힘 의원 경남 창원시 성산구 보건복지위원회 간사 이 오는 일 목 오후 국회의원회관 제 간담회의실에서 코로나 재유행 의료체계 점검 긴급간담회 를 개최할 예정이다 이번 간담회는 후반기 국회 보건복지위 정춘숙 한정애 의원 이끈다보건복지위 간사로는 국민의힘 의원과 더불어민주당 강훈식 의원이 각각 내정됐다 의원은 전반기에서도 보건복지위원회 국민의힘 간사로 활동했다 지난 월 일 일찍이 보건복지위 국민의힘 간사로 코로나 재유행대비 의료체계 점검 위한 긴급 간담회 개최국민의힘 보건복지위원회 간사인 의원 경남 창원성산구 사진 국제뉴스 국회 국민의힘 보건복지위원회 간

# ★★★실행★★★

- 크롤링
    - 만든거 for 문으로 휘리릭
    - 과정과 결과를 딕셔너리  에 저장 {MPname: [df_org, df_r1, text]}
        - 객체로 저장하면 어떨까 MP_dict = {MPname : mp 객체}
        - 접근은 MPdict['강기윤'].df_r1 이렇게. 하면 편하겠다.

- 1차 전처리
    - 한글이외의 것 삭제
    - 검색어 삭제 (여기선 윤석열)
    - (title, description)으로 중복 거른다.

- 2차 전처리
    - 모든 title description을 하나의 문자열 text로 합친다.

In [339]:
class MPdata:
    def __init__(self, name):
        self.name = name
    def save_org(self, df_org):
        self.df_org = df_org
    def save_r1(self, df_r1):
        self.df_r1 = df_r1
        self.len = df_r1.count().max()
    def save_text(self, text):
        self.text = text

In [340]:
MPdict = {}
nMP = len(MPnames)
for i, name in enumerate(MPnames):
#     print('***** ', i, '/', nMP, ' *****')
    print(nMP - i, end=" ")

    mp = MPdata(name)
    
    df = getNewsAbout(name)
    mp.save_org(df)
    
    df_r1 = reduction1(df, name)
    mp.save_r1(df_r1)
    
    text = reduction2(df_r1)
    mp.save_text(text)
    
    MPdict[name] = mp

299 298 297 296 295 294 293 292 291 290 289 288 287 286 285 284 283 282 281 280 279 278 277 276 275 274 273 272 271 270 269 268 267 266 265 264 263 262 261 260 259 258 257 256 255 254 253 252 251 250 249 248 247 246 245 244 243 242 241 240 239 238 237 236 235 234 233 232 231 230 229 228 227 226 225 224 223 222 221 220 219 218 217 216 215 214 213 212 211 210 209 208 207 206 205 204 203 202 201 200 199 198 197 196 195 194 193 192 191 190 189 188 187 186 185 184 183 182 181 180 179 178 177 176 175 174 173 172 171 170 169 168 167 166 165 164 163 162 161 160 159 158 157 156 155 154 153 152 151 150 149 148 147 146 145 144 143 142 141 140 139 138 137 136 135 134 133 132 131 130 129 128 127 126 125 124 123 122 121 120 119 118 117 116 115 114 113 112 111 110 109 108 107 106 105 104 103 102 101 100 99 98 97 96 95 94 93 92 91 90 89 88 87 86 85 84 83 82 81 80 79 78 77 76 75 74 73 72 71 70 69 68 67 66 65 64 63 62 61 60 59 58 57 56 55 54 53 52 51 50 49 48 47 46 45 44 43 42 41 40 39 38 37 36 35 34 33

In [324]:
MPdict

{'강기윤': <__main__.MPdata at 0x1acc0a1aa70>,
 '강대식': <__main__.MPdata at 0x1acc0a2feb0>,
 '강득구': <__main__.MPdata at 0x1acc0a19c00>,
 '강민국': <__main__.MPdata at 0x1acc0a42920>,
 '강민정': <__main__.MPdata at 0x1acbafba440>,
 '강병원': <__main__.MPdata at 0x1acc217fb50>,
 '강선우': <__main__.MPdata at 0x1acc2522f50>,
 '강은미': <__main__.MPdata at 0x1acc2263490>,
 '강준현': <__main__.MPdata at 0x1acc2435810>,
 '강훈식': <__main__.MPdata at 0x1acc1fc33d0>,
 '고민정': <__main__.MPdata at 0x1acc1fc3340>,
 '고영인': <__main__.MPdata at 0x1acc1fc35e0>,
 '고용진': <__main__.MPdata at 0x1acc1fc3cd0>,
 '구자근': <__main__.MPdata at 0x1acc27988e0>,
 '권명호': <__main__.MPdata at 0x1acc27988b0>,
 '권성동': <__main__.MPdata at 0x1acc2798dc0>,
 '권영세': <__main__.MPdata at 0x1acc27996c0>,
 '권은희': <__main__.MPdata at 0x1acc27994b0>,
 '권인숙': <__main__.MPdata at 0x1acc2799930>,
 '권칠승': <__main__.MPdata at 0x1acc279a2c0>,
 '기동민': <__main__.MPdata at 0x1acc279a6e0>,
 '김경만': <__main__.MPdata at 0x1acc279aef0>,
 '김경협': <__main__.MPdata at 0x1a

# 파일로 저장

In [325]:
import pickle

In [341]:
num

4

In [342]:
# 파일로 뽑음
with open(f'MP_news_data_{num}.pkl', 'wb') as f:
    pickle.dump(MPdict, f)
    num += 1

In [302]:
# 객체를 저장하려고 피클을 사용

# 저장된 파일 열어서 확인

In [343]:
with open('MP_news_data_4.pkl', 'rb') as fr: 
    MPdict_r = pickle.load(fr)

In [344]:
MPdict_r['강기윤'].df_org

,title,originallink,link,description,pubDate
0,확진자 접촉 무증상자도 신...,http://www.edail...,https://n.news.n...,국민의힘 의원 사진 은 이...,"Fri, 29 Jul 2022..."
1,의원 코로나 무증상자도 신...,https://mdtoday....,https://mdtoday....,국민의힘 의원 사진 의원...,"Fri, 29 Jul 2022..."
2,보건복지위원장 정춘숙 간사 ...,https://www.dail...,https://www.dail...,사진설명 좌측부터 보건복지위...,"Fri, 22 Jul 2022..."
3,후반기 국회 복지위원장에 정춘...,http://www.medig...,http://www.medig...,여당 간사는 국민의힘 의원이...,"Fri, 22 Jul 2022..."
4,팬데믹 시대 과학 방역의 핵...,http://www.sisao...,http://www.sisao...,토론자 박향 정재훈 조동찬 ...,"Tue, 26 Jul 2022..."
...,...,...,...,...,...
91,부음 이승헌 장모,https://www.busi...,https://www.busi...,세명대 석좌교수 박창순 박창...,"Tue, 05 Apr 2022..."
93,부고 씨 국회의원 빙모상,http://www.fnnew...,https://n.news.n...,김연석씨 별세 씨 국회의원...,"Tue, 05 Apr 2022..."
94,부고 씨 국회의원 장모상,https://www.news...,https://n.news.n...,김연석씨 별세 씨 국회의원...,"Tue, 05 Apr 2022..."
97,의원과 인사하는 안철수 인...,http://yna.kr/PY...,https://n.news.n...,안철수 대통령직 인수위원장 오...,"Thu, 31 Mar 2022..."


In [345]:
MPdict_r['강기윤'].df_r1

,title,originallink,link,description,pubDate
0,확진자 접촉 무증상자도 신...,http://www.edail...,https://n.news.n...,국민의힘 의원 사진 은 이...,"Fri, 29 Jul 2022..."
1,의원 코로나 무증상자도 신...,https://mdtoday....,https://mdtoday....,국민의힘 의원 사진 의원...,"Fri, 29 Jul 2022..."
2,보건복지위원장 정춘숙 간사 ...,https://www.dail...,https://www.dail...,사진설명 좌측부터 보건복지위...,"Fri, 22 Jul 2022..."
3,후반기 국회 복지위원장에 정춘...,http://www.medig...,http://www.medig...,여당 간사는 국민의힘 의원이...,"Fri, 22 Jul 2022..."
4,팬데믹 시대 과학 방역의 핵...,http://www.sisao...,http://www.sisao...,토론자 박향 정재훈 조동찬 ...,"Tue, 26 Jul 2022..."
...,...,...,...,...,...
91,부음 이승헌 장모,https://www.busi...,https://www.busi...,세명대 석좌교수 박창순 박창...,"Tue, 05 Apr 2022..."
93,부고 씨 국회의원 빙모상,http://www.fnnew...,https://n.news.n...,김연석씨 별세 씨 국회의원...,"Tue, 05 Apr 2022..."
94,부고 씨 국회의원 장모상,https://www.news...,https://n.news.n...,김연석씨 별세 씨 국회의원...,"Tue, 05 Apr 2022..."
97,의원과 인사하는 안철수 인...,http://yna.kr/PY...,https://n.news.n...,안철수 대통령직 인수위원장 오...,"Thu, 31 Mar 2022..."


In [349]:
type(MPdict_r['강기윤'].len), MPdict_r['강기윤'].len

(numpy.int64, 89)

In [350]:
type(MPdict_r['강기윤'].text), MPdict_r['강기윤'].text

(str,
 '확진자 접촉 무증상자도 신속항원 검사 무료화 국민의힘 의원 사진 은 이르면 다음주부터 코로나 확진자와 접촉한 무증상자 또한 의료기관 의원은 코로나 확진자와 접촉한 무증상자에 대해서도 의사의 판단 하에 신속항원검사비를 무료로 의원 코로나 무증상자도 신속항원검사 무료로 받을 수 있어 국민의힘 의원 사진 의원실 제공 이르면 다음주부터 코로나 확진자와 접촉한 무증상자 또한 의료기관 등에서 의사의 판단 하에 신속항원검사 를 무료로 받게 된다 국회 보건복지위원회 소속 보건복지위원장 정춘숙 간사 강훈식 의원 사진설명 좌측부터 보건복지위원장 정춘숙 의원 국민의힘 의원 더민주당 강훈식 의원 보건복지위 위원장은 보건복지위원장 정춘숙 의원이 맡았으며 간사는 여당에서 의원 국민의힘 후반기 국회 복지위원장에 정춘숙 의원 양당 간사는 강훈식여당 간사는 국민의힘 의원이 야당 간사는 더불어민주당 강훈식 의원이 맡게됐다 의원은 전반기에 이어 후반기에도 간사 직책을 이어가게 됐으며 강훈식 의원은 전반기 팬데믹 시대 과학 방역의 핵심 관료 아닌 전문가에 맡길 것 안철수 민 당 토론자 박향 정재훈 조동찬 국민의힘 안철수 의원이 일 오전 국회 의원회관에서 열린 위기를 연합뉴스 제언 전문가 의사 결정 따른 과학방역 이뤄야 국가 책임 중요 국민의힘 의원은 코로나 재유행대비 의료체계 점검 긴급 간담회 연다 의원 국회 국민의힘 의원 경남 창원시 성산구 보건복지위원회 간사 이 오는 일 목 오후 국회의원회관 제 간담회의실에서 코로나 재유행 의료체계 점검 긴급간담회 를 개최할 예정이다 이번 간담회는 후반기 국회 보건복지위 정춘숙 한정애 의원 이끈다보건복지위 간사로는 국민의힘 의원과 더불어민주당 강훈식 의원이 각각 내정됐다 의원은 전반기에서도 보건복지위원회 국민의힘 간사로 활동했다 지난 월 일 일찍이 보건복지위 국민의힘 간사로 코로나 재유행대비 의료체계 점검 위한 긴급 간담회 개최국민의힘 보건복지위원회 간사인 의원 경남 창원성산구 사진 국제뉴스 국회 국민의힘 보건복지위원회 간사인 의원 창원시 성산

In [348]:
MPdict_r['강기윤'].name

'강기윤'

# 테스트

## 네이버 API

- https://developers.naver.com/docs/serviceapi/search/news/news.md#%EB%89%B4%EC%8A%A4

In [184]:
client_id = "ISdXTGuV8Tdqx9fNxPgz"
client_secret = "e2kcexi37t"

encText = urllib.parse.quote("윤석열")
display = 100 # 출력건수 (최대)
sort = 'sim' # 유사도

In [185]:
def getDFabout(cat):
    url = f'https://openapi.naver.com/v1/search/{cat}?query={encText}&display={display}&sort={sort}'

    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)

    response = urllib.request.urlopen(request)
    rescode = response.getcode()

    if(rescode==200):
        response_body = response.read()
        print('Success')
    else:
        print("Error Code:" + rescode)
    data = response_body.decode('utf-8')
    data=json.loads(data)
    data1 = pd.DataFrame(data['items'])
    return data1

In [186]:
df = getDFabout('news')

Success


In [187]:
pd.options.display.max_colwidth = 20

In [188]:
df.columns

Index(['title', 'originallink', 'link', 'description', 'pubDate'], dtype='object')

In [189]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         100 non-null    object
 1   originallink  100 non-null    object
 2   link          100 non-null    object
 3   description   100 non-null    object
 4   pubDate       100 non-null    object
dtypes: object(5)
memory usage: 4.0+ KB


In [190]:
df

,title,originallink,link,description,pubDate
0,1년 형·언니와 경쟁 등떠밀린...,https://www.hani...,https://n.news.n...,사회적 논란 속에 사실상 폐기...,"Fri, 29 Jul 2022..."
1,[속보] <b>윤석열</b> ...,https://www.hank...,https://n.news.n...,<b>윤석열</b> 정부의 첫...,"Thu, 28 Jul 2022..."
2,경란(警亂) 고비 넘긴 <b>...,http://www.sisaj...,https://n.news.n...,<b>윤석열</b> 정부의 승...,"Fri, 29 Jul 2022..."
3,<b>윤석열</b> 대통령 직...,https://www.khan...,https://n.news.n...,<b>윤석열</b> 대통령의 ...,"Fri, 29 Jul 2022..."
4,김정은 “위험한 시도 시 <b...,https://news.kbs...,https://n.news.n...,북한 김정은 위원장이 &apo...,"Thu, 28 Jul 2022..."
...,...,...,...,...,...
95,"[전문] 김정은, 尹대통령 실...",https://biz.chos...,https://n.news.n...,"김정은, 연설서 “<b>윤석열...","Thu, 28 Jul 2022..."
96,직접 소통하던 <b>윤석열</...,https://www.hani...,https://n.news.n...,<b>윤석열</b> 대통령의 ...,"Thu, 28 Jul 2022..."
97,국기에 경례하는 <b>윤석열<...,https://www.yna....,https://n.news.n...,<b>윤석열</b> 대통령과 ...,"Thu, 28 Jul 2022..."
98,&quot;진작 탈당하길 잘했...,http://www.edail...,https://n.news.n...,이는 앞서 <b>윤석열</b>...,"Thu, 28 Jul 2022..."


### description 내용 확인

In [191]:
df.description

0     사회적 논란 속에 사실상 폐기...
1     <b>윤석열</b> 정부의 첫...
2     <b>윤석열</b> 정부의 승...
3     <b>윤석열</b> 대통령의 ...
4     북한 김정은 위원장이 &apo...
             ...         
95    김정은, 연설서 “<b>윤석열...
96    <b>윤석열</b> 대통령의 ...
97    <b>윤석열</b> 대통령과 ...
98    이는 앞서 <b>윤석열</b>...
99    배 최고위원은 “<b>윤석열<...
Name: description, Length: 100, dtype: object

In [192]:
description_len = df.description.apply(lambda x : len(x))

In [193]:
description_len.describe() # 길이도.. 제각각...

count    100.000000
mean     128.330000
std       21.629131
min       81.000000
25%      116.750000
50%      135.000000
75%      140.250000
max      166.000000
Name: description, dtype: float64

In [194]:
description_len

0     137
1     131
2     158
3     129
4     166
     ... 
95    139
96    138
97     91
98    130
99    124
Name: description, Length: 100, dtype: int64

In [195]:
desc = df.description[95]

In [196]:
for d in desc:
    print(d, end="")

김정은, 연설서 “<b>윤석열</b>이 내뱉은 망언과 추태 정확히 기억하고 있다” 국가안보실 “깊은 유감” 김정은... 조선중앙통신에 따르면 김정은은 연설에서 <b>윤석열</b> 정부 출범 이후 처음으로 직접 <b>윤석열</b> 대통령 이름을... 

In [197]:
desc[len(desc)-5:] # ...은 출력이 생략된게 아니라 이런 문자열이었음 젠장

'을... '

In [198]:
for i, desc in enumerate(df.description):
    print("*"*10, i, "*"*10)
    for d in desc:
        print(d, end="")
    print()

********** 0 **********
사회적 논란 속에 사실상 폐기된 정책을 <b>윤석열</b> 정부가 다시 꺼내들었다. 1949년 최초 제정된 교육법... 이른바 ‘<b>윤석열</b> 세대’ ‘박순애 세대’의 탄생이다. 박순애 사회부총리 겸 교육부장관은 29일 초등학교 입학... 
********** 1 **********
<b>윤석열</b> 정부의 첫 대법관 후보로 오석준(60·사법연수원 19기) 제주지방법원장이 선정됐다. 대법원은 김명수 대법원장이 <b>윤석열</b> 대통령에게 오는 9월 퇴임 예정인 김재형 대법관의 후임으로 오 법원장의 임명을... 
********** 2 **********
<b>윤석열</b> 정부의 승리, 경찰의 패배라고 말할 수 있을까. 모두가 졌다. 엠브레인퍼블릭, 케이스탯리서치... <b>윤석열</b> 대통령 국정운영에 대한 부정평가는 54%로 집계됐는데, &apos;독단적이고 일방적이어서&apos;가 30%를 차지했다. &quot;행안부... 
********** 3 **********
<b>윤석열</b> 대통령의 직무수행 긍정율이 28%를 기록했다는 여론조사결과가 29일 나왔다. 취임 80일 만에 직무수행 긍정률이 30% 밑으로 떨어졌다. 부정평가가 긍정평가를 웃도는 ‘데드크로스’에 이어 30% 지지율마저... 
********** 4 **********
북한 김정은 위원장이 &apos;선제 타격&apos; 등 <b>윤석열</b> 정부 대북 정책에 대해 강력하게 반발하고 나섰습니다. 김... 연설에 나선 김 위원장은 &apos;선제 타격&apos; 등 <b>윤석열</b> 정부의 정책과 관련해 &quot;그러한 위험한 시도는 즉시 강력한 힘에... 
********** 5 **********
<b>윤석열</b> 대통령이 29일 서울 용산 대통령실 청사에서 열린 주한대사 신임장 제정식에서 에르덴촉트 사랑터거스 주한 몽골 대사로부터 신임장을 받은 뒤 차담하고 있다. (대통령실통신사진기자단) 2022.07.29. yesphot

<b>윤석열</b> 대통령이 29일 오전 서울 세종로 정부서울청사에서 열린 코로나19 중앙재난안전대책본부 회의에 입장하며 이상민 행정안전부 장관과 악수하고 있다. (대통령실통신사진기자단) 2022.07.29. yesphoto@newsis.com... 
********** 68 **********
<b>윤석열</b> 대통령이 29일 오전 서울 세종로 정부서울청사에서 열린 코로나19 중앙재난안전대책본부 회의에 참석해 이상민 행정안전부 장관과 대화하고 있다. (대통령실통신사진기자단) 2022.07.29. yesphoto@newsis.com... 
********** 69 **********
<b>윤석열</b> 대통령과 김건희 여사가 28일 오전 울산시 현대중공업에서 열린 차세대 이지스구축함 정조대왕함 진수식에 입장하고 있다. 2022.7.28
********** 70 **********
<b>윤석열</b> 대통령이 29일 오전 서울 서대문구 신촌지구대를 방문, 경찰들과 업무 관련 대화를 하고 있다. 2022.7.29 [대통령실 제공. 재판매 및 DB금지]
********** 71 **********
<b>윤석열</b> 대통령과 김건희 여사가 28일 오전 울산시 현대중공업에서 열린 차세대 이지스구축함 정조대왕함 안전항해 기원식을 마친 뒤 장병들을 향해 손을 흔들고 있다. 2022.7.28
********** 72 **********
<b>윤석열</b> 대통령이 29일 오전 서울 세종로 정부서울청사에서 열린 코로나19 중앙재난안전대책본부 회의를 주재하고 있다. 2022.7.29
********** 73 **********
<b>윤석열</b> 대통령이 29일 오전 서울 서대문구 신촌지구대를 방문, 폭염 속 민생치안 활동에 여념이 없는 경찰관들과 환담에 앞서 인사하고 있다. 2022.7.29.
********** 74 **********
<b>윤석열</b> 대통령과 김건희 여사가 28일 오전 울산시 현대중공업에서 열린 차세대 이지스구축함 정조대왕함 안전항해 기원식을 마친 

In [199]:
df.description[0]

'사회적 논란 속에 사실상 폐기된 정책을 <b>윤석열</b> 정부가 다시 꺼내들었다. 1949년 최초 제정된 교육법... 이른바 ‘<b>윤석열</b> 세대’ ‘박순애 세대’의 탄생이다. 박순애 사회부총리 겸 교육부장관은 29일 초등학교 입학... '

In [202]:
print(df.link[0])

https://n.news.naver.com/mnews/article/028/0002600546?sid=102


In [206]:
# 본문에서 검색어의 단어가 포함된 문장(+뒷 문장 일부)을 가져온다. 2번째 일치 되는 것까지만..

## 전처리

### 1차
- 한글이외의 것 삭제
- 검색어 삭제 (여기선 윤석열)
- title, description으로 중복 거른다.

In [40]:
df.drop_duplicates(subset=['title']).drop_duplicates(subset=['description']).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79 entries, 0 to 99
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         79 non-null     object
 1   originallink  79 non-null     object
 2   link          79 non-null     object
 3   description   79 non-null     object
 4   pubDate       79 non-null     object
dtypes: object(5)
memory usage: 3.7+ KB


In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         100 non-null    object
 1   originallink  100 non-null    object
 2   link          100 non-null    object
 3   description   100 non-null    object
 4   pubDate       100 non-null    object
dtypes: object(5)
memory usage: 4.0+ KB


### 2차
- 모든 title description을 하나의 문자열 text로 합친다.

In [218]:
sr = df.title + df.description
sr.info(), sr[0], type(sr), type(sr[0])

<class 'pandas.core.series.Series'>
RangeIndex: 100 entries, 0 to 99
Series name: None
Non-Null Count  Dtype 
--------------  ----- 
100 non-null    object
dtypes: object(1)
memory usage: 928.0+ bytes


(None,
 '1년 형·언니와 경쟁 등떠밀린 ‘<b>윤석열</b> 세대’ 탄생하나사회적 논란 속에 사실상 폐기된 정책을 <b>윤석열</b> 정부가 다시 꺼내들었다. 1949년 최초 제정된 교육법... 이른바 ‘<b>윤석열</b> 세대’ ‘박순애 세대’의 탄생이다. 박순애 사회부총리 겸 교육부장관은 29일 초등학교 입학... ',
 pandas.core.series.Series,
 str)

In [220]:
sr

0     1년 형·언니와 경쟁 등떠밀린...
1     [속보] <b>윤석열</b> ...
2     경란(警亂) 고비 넘긴 <b>...
3     <b>윤석열</b> 대통령 직...
4     김정은 “위험한 시도 시 <b...
             ...         
95    [전문] 김정은, 尹대통령 실...
96    직접 소통하던 <b>윤석열</...
97    국기에 경례하는 <b>윤석열<...
98    &quot;진작 탈당하길 잘했...
99    배현진 &quot;<b>윤석열...
Length: 100, dtype: object

In [221]:
" ".join(sr)

'1년 형·언니와 경쟁 등떠밀린 ‘<b>윤석열</b> 세대’ 탄생하나사회적 논란 속에 사실상 폐기된 정책을 <b>윤석열</b> 정부가 다시 꺼내들었다. 1949년 최초 제정된 교육법... 이른바 ‘<b>윤석열</b> 세대’ ‘박순애 세대’의 탄생이다. 박순애 사회부총리 겸 교육부장관은 29일 초등학교 입학...  [속보] <b>윤석열</b> 정부 첫 대법관 후보에 오석준 임명 제청<b>윤석열</b> 정부의 첫 대법관 후보로 오석준(60·사법연수원 19기) 제주지방법원장이 선정됐다. 대법원은 김명수 대법원장이 <b>윤석열</b> 대통령에게 오는 9월 퇴임 예정인 김재형 대법관의 후임으로 오 법원장의 임명을...  경란(警亂) 고비 넘긴 <b>윤석열</b> 정부, 다음 수순은 경찰대 개혁?<b>윤석열</b> 정부의 승리, 경찰의 패배라고 말할 수 있을까. 모두가 졌다. 엠브레인퍼블릭, 케이스탯리서치... <b>윤석열</b> 대통령 국정운영에 대한 부정평가는 54%로 집계됐는데, &apos;독단적이고 일방적이어서&apos;가 30%를 차지했다. &quot;행안부...  <b>윤석열</b> 대통령 직무수행 긍정률 28%···30%선 붕괴[한국갤럽]<b>윤석열</b> 대통령의 직무수행 긍정율이 28%를 기록했다는 여론조사결과가 29일 나왔다. 취임 80일 만에 직무수행 긍정률이 30% 밑으로 떨어졌다. 부정평가가 긍정평가를 웃도는 ‘데드크로스’에 이어 30% 지지율마저...  김정은 “위험한 시도 시 <b>윤석열</b> 정권·군대 전멸될 것”북한 김정은 위원장이 &apos;선제 타격&apos; 등 <b>윤석열</b> 정부 대북 정책에 대해 강력하게 반발하고 나섰습니다. 김... 연설에 나선 김 위원장은 &apos;선제 타격&apos; 등 <b>윤석열</b> 정부의 정책과 관련해 &quot;그러한 위험한 시도는 즉시 강력한 힘에...  주한 몽골 대사와 차담하는 <b>윤석열</b> 대통령<b>윤석열</b> 대통령이 29일 서울 용산 대통령실 청사에서 열린 주한대사 신임장

# 시간되면 처리

## 단어로 짜갠다. word의 리스트words가 만들어진다.

## wordWhere = {word :  list_of_index} 이런 딕셔너리 만든다.

In [42]:
text = '강아지 으르릉 멍멍 강아지 악 강아지 강아아앙 강아쥐 갱얼쥐 강아지 강쥐'

In [43]:
words = text.split()

In [44]:
words

['강아지', '으르릉', '멍멍', '강아지', '악', '강아지', '강아아앙', '강아쥐', '갱얼쥐', '강아지', '강쥐']

In [45]:
import numpy as np

In [46]:
words = np.array(words)
words

array(['강아지', '으르릉', '멍멍', '강아지', '악', '강아지', '강아아앙', '강아쥐', '갱얼쥐', '강아지',
       '강쥐'], dtype='<U4')

In [47]:
np.where(words == '강아지')

(array([0, 3, 5, 9], dtype=int64),)

In [48]:
words

array(['강아지', '으르릉', '멍멍', '강아지', '악', '강아지', '강아아앙', '강아쥐', '갱얼쥐', '강아지',
       '강쥐'], dtype='<U4')

- 형태소 분석
- 조사 삭제

In [49]:
from konlpy.tag import Okt

In [50]:
okt = Okt()
morph = okt.pos(text)

In [51]:
text

'강아지 으르릉 멍멍 강아지 악 강아지 강아아앙 강아쥐 갱얼쥐 강아지 강쥐'

In [52]:
morph

[('강아지', 'Noun'),
 ('으르릉', 'Noun'),
 ('멍멍', 'Adverb'),
 ('강아지', 'Noun'),
 ('악', 'Noun'),
 ('강아지', 'Noun'),
 ('강', 'Noun'),
 ('아', 'Josa'),
 ('아앙', 'Noun'),
 ('강', 'Noun'),
 ('아', 'Josa'),
 ('쥐', 'Noun'),
 ('갱얼쥐', 'Noun'),
 ('강아지', 'Noun'),
 ('강쥐', 'Noun')]

In [62]:
# 의미를 모르고 쪼개는 것 같다.
# 한국어는 같은 단어에 전혀 다른 뜻이 쓰이는 경우가 많다.
# ex) 강 river strong/ 말 hourse words/ 눈 snow, eyes
# 의미 분석을 하려면 영어로 번역후 하는게 좋지 않을까?
# 네이버 파파고 api 활용 가능할지도

- Okt의 attr, method??
- jki morphs nomalize nouns phrases pos tagset

In [60]:
okt.jki

<java object 'kr.lucypark.okt.OktInterface'>

In [53]:
okt.morphs(text)

['강아지',
 '으르릉',
 '멍멍',
 '강아지',
 '악',
 '강아지',
 '강',
 '아',
 '아앙',
 '강',
 '아',
 '쥐',
 '갱얼쥐',
 '강아지',
 '강쥐']

In [54]:
okt.tagset

{'Adjective': '형용사',
 'Adverb': '부사',
 'Alpha': '알파벳',
 'Conjunction': '접속사',
 'Determiner': '관형사',
 'Eomi': '어미',
 'Exclamation': '감탄사',
 'Foreign': '외국어, 한자 및 기타기호',
 'Hashtag': '트위터 해쉬태그',
 'Josa': '조사',
 'KoreanParticle': '(ex: ㅋㅋ)',
 'Noun': '명사',
 'Number': '숫자',
 'PreEomi': '선어말어미',
 'Punctuation': '구두점',
 'ScreenName': '트위터 아이디',
 'Suffix': '접미사',
 'Unknown': '미등록어',
 'Verb': '동사'}

In [55]:
okt.pos('코로나19')

[('코로나', 'Noun'), ('19', 'Number')]

In [56]:
okt.pos('시발아')

[('시발', 'Noun'), ('아', 'Josa')]

In [57]:
okt.pos('완전 짜증 났잖아~~ 아오')

[('완전', 'Noun'),
 ('짜증', 'Noun'),
 ('났잖아', 'Verb'),
 ('~~', 'Punctuation'),
 ('아오', 'Exclamation')]

In [58]:
text

'강아지 으르릉 멍멍 강아지 악 강아지 강아아앙 강아쥐 갱얼쥐 강아지 강쥐'

In [59]:
okt.phrases(text)

['강아지',
 '강아지 으르릉',
 '강아지 악',
 '강아지 악 강아지',
 '강아지 악 강아지 강',
 '아앙',
 '아앙 강',
 '쥐 갱얼쥐',
 '쥐 갱얼쥐 강아지',
 '쥐 갱얼쥐 강아지 강쥐',
 '으르릉',
 '갱얼쥐',
 '강쥐']

- 이건 안해도 될 것 같지만 하면 좋겠다.
    - 빈도 확인
    - 막대그래프랑 워드클라우드 살짝 뽑아보기